In [311]:
import cvxpy as cp
import numpy as np
from numpy.typing import NDArray
import random

def get_ordered_comparison_model(line_array : NDArray, epsilon=1e-3, M = 1e8):
    ordered_list = get_list_of_arrays_less_than_k(line_array)
    Cis = cp.Variable(line_array.shape[0], boolean=True, name=f'c{random.randrange(1,1000)}')
    constraints_hi = []
    for idx in range(len(ordered_list)):
        constraints_hi += [ordered_list[idx][k] <= ordered_list[idx][idx] + epsilon - M*(1-Cis[idx])
                          for k in range(ordered_list[idx].shape[0]-1)]
    return Cis, constraints_hi

def get_list_of_arrays_less_than_k(line_array : NDArray):
    return [line_array[:k+1] for k in range(line_array.shape[0])]

# Example data
A = np.array([2, 3, 1, 4]) 
print(get_list_of_arrays_less_than_k(A))
epsilon = 1e-5  # Tolerance for strict inequality
M = 1e5  # Big-M value

Cis, constraints = get_ordered_comparison_model(A, epsilon, M)
objective = cp.Maximize(cp.sum(Cis))
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.SCIP)

print(problem.status)

# Results
print(f"Number of towers to see: {problem.value}")

[array([2]), array([2, 3]), array([2, 3, 1]), array([2, 3, 1, 4])]
infeasible
Number of towers to see: -inf


In [309]:
import cvxpy as cp

def make_discrete_decision_variables(array_shape, allowed_values):
    # Binary variables: z[i,j,k] = 1 if C[i,j] == allowed_values[k]
    z = cp.Variable( array_shape + tuple([len(allowed_values)]), boolean=True, name='z')
    constraints = []
    for i in range(array_shape[0]):
        for j in range(array_shape[1]):
            # Only one value can be chosen per C[i,j]
            constraints += [cp.sum(z[i, j, :]) == 1]
    return z, constraints

def guarantee_one_of_each_in_full_matrix(z_array, allowed_values):
    constraints = []
    # for each column (it is supposed to be switched like that)
    for row in range(z_array.shape[0]):
        # Each value v must appear at least once in C
        for v_idx, _ in enumerate(allowed_values):
            constraints += [cp.sum(z_array[row,:, v_idx]) == 1]  # one occurrence of v
    
    # for each row (it is supposed to be switched like that)
    for column in range(z_array.shape[0]):
        # Each value v must appear at least once in C
        for v_idx, _ in enumerate(allowed_values):
            constraints += [cp.sum(z_array[:,column, v_idx]) == 1]  # one occurrence of v
    return constraints

def get_final_solution_constraints(decision_variables):
    final_result = cp.Variable(decision_variables[:,:,0].shape, integer=True, name='final')
    final_result_arr = np.zeros(decision_variables[:,:,0].shape)

    for idx in range(decision_variables.shape[2]):
        final_result_arr = final_result_arr + decision_variables[:,:, idx]*allowed_values[idx]

    constraint = [final_result == final_result_arr]
    
    return final_result, constraint

n = 4
allowed_values = [1, 2, 3, 4]  # Discrete set
decision_variables, constraints = make_discrete_decision_variables((n, n), allowed_values)
constraints += guarantee_one_of_each_in_full_matrix(decision_variables, allowed_values)

final_solution, final_solution_constraints = get_final_solution_constraints(decision_variables)
constraints += final_solution_constraints

z = decision_variables

# fill some just to test
test_contraints = [z[1, 1, 2] == 1, z[1, 2, 1] == 1, z[1, 3, 0] == 1, z[2, 3, 2] == 1,
                   z[3, 1, 3] == 1, z[0, 1, 0] == 1]
constraints += test_contraints

objective = cp.Maximize(cp.sum(decision_variables))

problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.SCIP)

print(problem.status)
print(final_solution.value)


optimal
[[2. 1. 3. 4.]
 [4. 3. 2. 1.]
 [1. 2. 4. 3.]
 [3. 4. 1. 2.]]


In [310]:
n = 4
constraints = []

side_towers = np.zeros((n, 2, 2)) #position, row or column, orientation (right or left, down or up)

def get_side_towers_constraints(final_result_vars, side_towers):
    constraints = []
    for position in range(side_towers.shape[0]):
        for pos_type in range(side_towers.shape[1]):
            for orientation in range(side_towers.shape[2]):
                if pos_type == 0:
                    if orientation == 0:
                        cis, tower_constraint = get_ordered_comparison_model(final_result_vars[position, :])
                    else:
                        cis, tower_constraint = get_ordered_comparison_model(final_result_vars[position,::-1])
                else:
                    if orientation == 0:
                        cis, tower_constraint = get_ordered_comparison_model(final_result_vars[:, position])
                    else:
                        cis, tower_constraint = get_ordered_comparison_model(final_result_vars[::-1, position])
                      
                towers_value = side_towers[position, pos_type, orientation]
                if not(towers_value == 0):
                    constraint1 = tower_constraint.copy()
                    print([str(c) for c in constraint1])
                    constraints += tower_constraint.copy()
                    constraint2 = [towers_value <= cp.sum(cis)]
                    print(str(constraint2[0]))
                    constraints += [towers_value <= cp.sum(cis)]
        
    return constraints

## Test
s = side_towers
## Linhas
s[1,0,0] = 2
s[3,0,1] = 4
#s[2,0,0] = 1
#s[3,0,0] = 2
## Colunas 
s[1,1,0] = 3
s[0,1,0] = 1
s[3,1,1] = 2


#s[2,0,0] = 4
#s[3,1,0] = 2
side_towers = s

#print(side_towers)

n = 4
allowed_values = [1, 2, 3, 4]  # Discrete set
decision_variables, constraints = make_discrete_decision_variables((n, n), allowed_values)
constraints += guarantee_one_of_each_in_full_matrix(decision_variables, allowed_values)

final_solution, final_solution_constraints = get_final_solution_constraints(decision_variables)
constraints += final_solution_constraints

tower_constraints = get_side_towers_constraints(final_solution, side_towers)
constraints += tower_constraints

objective = cp.Maximize(cp.sum(decision_variables))

problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.SCIP)

print(problem.status)
for vars in problem.variables()[2:10]:
    print(f"{vars} : {vars.value}")
print(final_solution.value)


['final[0:4, 0][0:2][0] + 0.001 + -100000000.0 @ (1.0 + -c44[1]) <= final[0:4, 0][0:2][1]', 'final[0:4, 0][0:3][0] + 0.001 + -100000000.0 @ (1.0 + -c44[2]) <= final[0:4, 0][0:3][2]', 'final[0:4, 0][0:3][1] + 0.001 + -100000000.0 @ (1.0 + -c44[2]) <= final[0:4, 0][0:3][2]', 'final[0:4, 0][0:4][0] + 0.001 + -100000000.0 @ (1.0 + -c44[3]) <= final[0:4, 0][0:4][3]', 'final[0:4, 0][0:4][1] + 0.001 + -100000000.0 @ (1.0 + -c44[3]) <= final[0:4, 0][0:4][3]', 'final[0:4, 0][0:4][2] + 0.001 + -100000000.0 @ (1.0 + -c44[3]) <= final[0:4, 0][0:4][3]']
1.0 <= Sum(c44, None, False)
['final[1, 0:4][0:2][0] + 0.001 + -100000000.0 @ (1.0 + -c549[1]) <= final[1, 0:4][0:2][1]', 'final[1, 0:4][0:3][0] + 0.001 + -100000000.0 @ (1.0 + -c549[2]) <= final[1, 0:4][0:3][2]', 'final[1, 0:4][0:3][1] + 0.001 + -100000000.0 @ (1.0 + -c549[2]) <= final[1, 0:4][0:3][2]', 'final[1, 0:4][0:4][0] + 0.001 + -100000000.0 @ (1.0 + -c549[3]) <= final[1, 0:4][0:4][3]', 'final[1, 0:4][0:4][1] + 0.001 + -100000000.0 @ (1.0 + 